# Performance of different cosine similarity implementations

### Intro

Cosine similarity is widely used in text comparision tasks. The texts are first turned into a term frequency matrix (TF), or TF-IDF. In such a form, each text is a vector in the vocabulary space. When two texts use a lot of common words with similar frequency, their vectors are nearly collinear, and cosine of the angle between them is close to 1.  On the opposite side, when two texts' words are mostly different, the vectors are ortogonal and cosine is close to 0. The formulae for cosine difference is the following:

\begin{equation}cos(\pmb x, \pmb y) = \frac {\pmb x \cdot \pmb y}{||\pmb x|| \cdot ||\pmb y||}\end{equation}

The usual task in recommendation system is to find texts most similar to a given one. For example, a job board site may want to display few similar vacancies to a user who applied to a particular job posting. 

In an offline batch, we can compute cosine similarity of every text to every other and look up for vacancies that are most similar to few interested to our user.

Let's look at several possible implmentations of the cosine similarity.

We'll be measuring execution speed with a help of a wrapper around Python's timeit().

In [1]:
import timeit
from collections import OrderedDict

In [2]:
timeit.template = """
def inner(_it, _timer{init}):
    {setup}
    _t0 = _timer()
    for _i in _it:
        retval = {stmt}
    _t1 = _timer()
    return _t1 - _t0, retval
"""

In [3]:
dict = OrderedDict() #keep all measurements here
def my_timeit(dict,f,n=10,fname=""):
    fname = f.__name__ 
    dict_name = fname if fname == "" else fname
    #res = timeit.timeit('s='+fname+'(M)',setup="from __main__ import "+fname+", M", number=n)
    res, s = timeit.timeit(fname+'(M)',globals=globals(), number=n)
    
    dict[dict_name] = res/n
    print('{0:.3f}'.format(dict[dict_name]))
    return s

### The data
For illustration purpose, we are not going to process actual texts, we start from a TF-IDF matrix straing away.
Let's generate a matrix similar to what TfidfVectorizer would give us, converted to dense format.

In [4]:
from scipy import sparse

In [5]:
n_texts = 1018
n_words = 14667
density = 0.07

M = sparse.rand(n_texts, n_words, density).toarray()

In [6]:
#the matrix is sparse, but we use it in a dense format. This will work fine for small set of texts
M

array([[0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      ],
       ...,
       [0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , ..., 0.      , 0.466119, 0.      ]])

### Naive approach

This is what some one with programming experience, butlittle exposure to python and numpy would come with.
Since we need to compare al text to all other texts, we just loop over all texts, and inside that loop we loop again over the remaining text and compute the cosine according to the formula above.
Note, that we do not attempt to implement scalar product or vectors norm from scratch - this would not make sence even for a beginner!

In [7]:
import numpy as np
from numpy.linalg import norm

In [8]:
def cos_matrix(M):
    s = np.zeros((len(M),len(M)))

    for i in range(len(M)):
        iv=M[i]
        for j in range(i, len(M)):
            jv=M[j]
            s[i,j] = np.dot(iv,jv)/norm(iv)/norm(jv)
    return s

In [9]:
s = my_timeit(dict, cos_matrix, n=3, fname="naive")

12.373


In [10]:
s

array([[1.        , 0.03935207, 0.05498267, ..., 0.05346561, 0.04176018,
        0.05739019],
       [0.        , 1.        , 0.05328704, ..., 0.04680356, 0.05572377,
        0.03870978],
       [0.        , 0.        , 1.        , ..., 0.05307461, 0.05030439,
        0.04253165],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.06036136,
        0.04548421],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.05332369],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [11]:
dict

OrderedDict([('cos_matrix', 12.372514069080353)])

Side note: one can also use `%timeit`  magic allows as to see exactly how much time statement excution took:

```
%timeit -o s = cos_matrix(M)
1 loop, best of 3: 20.2 s per loop
```

We will ue it when our helper function is not practical

Here is the resulting matrix. 1s on the diagonal is expected, as well as 0s below the diagonal.

In [12]:
s.shape

(1018, 1018)

### Joblib - parallel execution
The second step one could think of is parallel execution. Since computin gcosine difference takes two vertos at a time, it should be trivial to scale it to a number of cores on the box/

Python gives a convinient way to parallelize a task via `joblib` package. 

To start with `joblib` we rewrite our function - the outer loop will be executed `len(M)` times by `joblib` itself, and a resulting vector ofsize `len(M)` will be returned back. Hence we get the same `s` matrix.

In [13]:
from joblib import Parallel, delayed

def cos_matrix_p(i):
    t = np.zeros(len(M))
    iv=M[i]
    for j in range(i,len(M)):
        jv=M[j]
        t[j] = np.dot(iv,jv)/norm(iv)/norm(jv)
    return t

In [14]:
def run_cos_matrix_p_threading(M):
    return Parallel(n_jobs=-1, verbose=1, backend="threading")( map(delayed(cos_matrix_p), range(0,len(M))) )

In [15]:
s = my_timeit(dict, run_cos_matrix_p_threading, n=3, fname="joblib_threading")

[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:   19.0s
[Parallel(n_jobs=-1)]: Done 338 tasks      | elapsed:   47.0s
[Parallel(n_jobs=-1)]: Done 688 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 1018 out of 1018 | elapsed:  1.4min finished
[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:   26.1s
[Parallel(n_jobs=-1)]: Done 338 tasks      | elapsed:   55.0s
[Parallel(n_jobs=-1)]: Done 688 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 1018 out of 1018 | elapsed:  1.5min finished
[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:   23.5s
[Parallel(n_jobs=-1)]: Done 338 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 688 tasks      | elapsed:  2.0min


101.850


[Parallel(n_jobs=-1)]: Done 1018 out of 1018 | elapsed:  2.2min finished


In [16]:
#res = %timeit -r 2 -o s = Parallel(n_jobs=-1, verbose=1, backend="threading")( map(delayed(cos_matrix_p), range(0,len(M))) )
#dict["joblib_threading"] = res.best

The result is dissapointing, but has explanation. Threading in python still uses interpretor and has the GIL as a bottleneck. Effectively, we still use a single OS thread during execution. Thus example is more like a cooperative multitasking than true threading. Besides, there an overhead of passing the data between threads.

Let's change the `joblib` backend to 'multiprocessing' that forks separate processes for each parallel task. This way we are sure to use more that one core of a host machine. But bear in mind that overhead of pasing the data between processes will even larger.

In [17]:
def run_cos_matrix_p_multiprocessing(M):
    return Parallel(n_jobs=-1, verbose=1, backend="multiprocessing")( map(delayed(cos_matrix_p), range(0,len(M))) )

In [18]:
#res = %timeit -r 3 -o s = Parallel(n_jobs=-1, verbose=1, backend="multiprocessing")( map(delayed(cos_matrix_p), range(0,len(M))))
#dict["joblib_multiprocessing"] = res.best

In [19]:
s = my_timeit(dict, run_cos_matrix_p_multiprocessing, n=3, fname="joblib_multiprocessing")

[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 338 tasks      | elapsed:    9.2s
[Parallel(n_jobs=-1)]: Done 688 tasks      | elapsed:   16.9s
[Parallel(n_jobs=-1)]: Done 1018 out of 1018 | elapsed:   19.0s finished
[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 338 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done 688 tasks      | elapsed:   13.0s
[Parallel(n_jobs=-1)]: Done 1018 out of 1018 | elapsed:   14.4s finished
[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done 449 tasks      | elapsed:    8.6s
[Parallel(n_jobs=-1)]: Done 799 tasks      | elapsed:   13.8s


16.793


[Parallel(n_jobs=-1)]: Done 1018 out of 1018 | elapsed:   14.9s finished


Here we got some real improvement! Multicore parallel processing is very useful.
Let's make sure the result is the same.

In [20]:
s[:5]

[array([1.        , 0.03935207, 0.05498267, ..., 0.05346561, 0.04176018,
        0.05739019]),
 array([0.        , 1.        , 0.05328704, ..., 0.04680356, 0.05572377,
        0.03870978]),
 array([0.        , 0.        , 1.        , ..., 0.05307461, 0.05030439,
        0.04253165]),
 array([0.        , 0.        , 0.        , ..., 0.04484612, 0.04931265,
        0.04804116]),
 array([0.        , 0.        , 0.        , ..., 0.04310505, 0.03313469,
        0.05046435])]

But can we still do better?

Let's check what our function spend time on. We use `%prun` to break down execution time per call.

In [21]:
result = %prun -r  cos_matrix(M)

         8818433 function calls in 14.155 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  1556013    7.118    0.000    7.118    0.000 {built-in method numpy.core.multiarray.dot}
  1037342    4.004    0.000   10.098    0.000 linalg.py:2103(norm)
        1    1.100    1.100   14.155   14.155 <ipython-input-8-3bec17bbceae>:1(cos_matrix)
  1037342    0.774    0.000    0.774    0.000 {method 'ravel' of 'numpy.ndarray' objects}
  1037342    0.313    0.000    0.313    0.000 {built-in method numpy.core.multiarray.array}
  2074684    0.309    0.000    0.309    0.000 {built-in method builtins.issubclass}
  1037342    0.280    0.000    0.593    0.000 numeric.py:424(asarray)
  1037342    0.249    0.000    0.374    0.000 linalg.py:110(isComplexType)
        1    0.008    0.008    0.008    0.008 {built-in method numpy.core.multiarray.zeros}
     1021    0.000    0.000    0.000    0.000 {built-in method builtins.len}
        1    0.000 

In [22]:
result.print_stats()

         8818433 function calls in 14.155 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  1556013    7.118    0.000    7.118    0.000 {built-in method numpy.core.multiarray.dot}
  1037342    4.004    0.000   10.098    0.000 linalg.py:2103(norm)
        1    1.100    1.100   14.155   14.155 <ipython-input-8-3bec17bbceae>:1(cos_matrix)
  1037342    0.774    0.000    0.774    0.000 {method 'ravel' of 'numpy.ndarray' objects}
  1037342    0.313    0.000    0.313    0.000 {built-in method numpy.core.multiarray.array}
  2074684    0.309    0.000    0.309    0.000 {built-in method builtins.issubclass}
  1037342    0.280    0.000    0.593    0.000 numeric.py:424(asarray)
  1037342    0.249    0.000    0.374    0.000 linalg.py:110(isComplexType)
        1    0.008    0.008    0.008    0.008 {built-in method numpy.core.multiarray.zeros}
     1021    0.000    0.000    0.000    0.000 {built-in method builtins.len}
        1    0.000 

Expectedly, `norm()` and `dot()` are the "winners".

### norm() vs. dot()*dot() 

Let's make a little trick. `norm()` is a just a square root or scalar product of a vector with itself. Let's get rid of `norm()` in favor of `dot()`

In [23]:
import math
def norm_dot(iv):
    return math.sqrt(np.dot(iv,iv))

In [24]:
def cos_matrix_dot_dot(M):
    s = np.zeros((len(M),len(M)))

    for i in range(len(M)):
        iv=M[i]
        for j in range(i,len(M)):
            jv=M[j]
            #s[i,j] = np.dot(iv,jv)/norm(iv)/norm(jv)
            s[i,j] = np.dot(iv,jv)/norm_dot(iv)/norm_dot(jv)
    return s

In [25]:
s = my_timeit(dict, cos_matrix_dot_dot, 3)

8.248


Quite unexpectedly, I'd say, we've got a solid improvement from the original run. Why? I don't know for sure. I might suspect that implementation of `dot()` is simpler than that of `norm()` since the latter is more universal - it can calculate a couple of dozens off dirrerent norms, and can also take 2D array of vectors as input. Hence some more general purpose code is executed in `norm()` than we really need in our example.

### Cache the norm with LRU

Next improvement we can think of is the following. Since we have to calculate `norm()` for the same vector over and over while looping, why not to compute `norm()` once for every vector and memorize the values?

Here we use Python's standard `lru_cache` that stands for Least Recently Used and is invoked as a decorator. Note the beauty of this approach - pretty much no code change!

In [26]:
from functools import lru_cache
# we have to use integer argument to norm_dot() this time since vectors are not hashable and can't be cache's keys.
@lru_cache(maxsize=None)
def norm_dot_i(i):
    iv = M[i] 
    return math.sqrt(np.dot(iv,iv))

In [27]:
def cos_matrix_dot_dot_cache(M):
    s = np.zeros((len(M),len(M)))

    for i in range(len(M)):
        iv=M[i]
        for j in range(i,len(M)):
            jv=M[j]
            #s[i,j] = np.dot(iv,jv)/norm(iv)/norm(jv)
            s[i,j] = np.dot(iv,jv)/norm_dot_i(i)/norm_dot_i(j)
    return s

In [28]:
s = my_timeit(dict, cos_matrix_dot_dot_cache, 3)

3.665


Expectedly better result. But we are still not on par with `joblib` multiprocessing.

## Let's try some library functions
Surely, some one must have done this task before and wanted to come up with an efficient implementaion?

## scipy pdist
The general task of calculating some function for every pair of given items is known as _pairwise calculation_.
ScyPy has a whole set of metrics that can be computed pairwise, and _cosine_ is among them.

In [29]:
from scipy.spatial.distance import pdist
def cos_matrix_scipy_pdist(M):
    return pdist(M, metric='cosine')

In [30]:
y_cosine = my_timeit(dict, cos_matrix_scipy_pdist, 3)

7.647


Pretty good! But the problem is that the result is flatten into a vector, that we have to somehow turn back to the matrix `s` that we can further use. Not so convinient.

In [31]:
len(y_cosine)

517653

In [32]:
((len(M)*len(M))-len(M))/2

517653.0

## Sklearn pairwise

Scikit-learn gives us two options. One is to calculate cosine similarity directly:

In [33]:
from sklearn.metrics.pairwise import cosine_similarity

In [34]:
def cos_matrix_sklearn_cossim(M):
    return cosine_similarity(M)

In [35]:
my_timeit(dict, cos_matrix_sklearn_cossim, 10)

0.173


array([[1.        , 0.03935207, 0.05498267, ..., 0.05346561, 0.04176018,
        0.05739019],
       [0.03935207, 1.        , 0.05328704, ..., 0.04680356, 0.05572377,
        0.03870978],
       [0.05498267, 0.05328704, 1.        , ..., 0.05307461, 0.05030439,
        0.04253165],
       ...,
       [0.05346561, 0.04680356, 0.05307461, ..., 1.        , 0.06036136,
        0.04548421],
       [0.04176018, 0.05572377, 0.05030439, ..., 0.06036136, 1.        ,
        0.05332369],
       [0.05739019, 0.03870978, 0.04253165, ..., 0.04548421, 0.05332369,
        1.        ]])

And voilà! We have a new record here! Lets try the other one and then discuss.

In [36]:
from sklearn.metrics.pairwise import pairwise_distances

In [37]:
def cos_matrix_sklearn_pd_cosine(M):
    return pairwise_distances(M, metric='cosine')

In [38]:
#res = %timeit -r 3 -o d = cos_matrix_sklearn_pd_cosine(M)
#dict["cos_matrix_sklearn_pd_cosine"] = res.best

In [39]:
d = my_timeit(dict, cos_matrix_sklearn_pd_cosine, 10)

0.164


In [40]:
#Cosine distance is defined as 1.0 minus cosine similarity.
1 - d[:5]

array([[1.        , 0.03935207, 0.05498267, ..., 0.05346561, 0.04176018,
        0.05739019],
       [0.03935207, 1.        , 0.05328704, ..., 0.04680356, 0.05572377,
        0.03870978],
       [0.05498267, 0.05328704, 1.        , ..., 0.05307461, 0.05030439,
        0.04253165],
       [0.06045495, 0.05640413, 0.05417889, ..., 0.04484612, 0.04931265,
        0.04804116],
       [0.06425379, 0.04670046, 0.06254431, ..., 0.04310505, 0.03313469,
        0.05046435]])

What we see here is that the less code we write - the better. It means that some task specific functions must have a lot of optimization inside, may even be implemented in _c_ and hence show significant improvement over pure Python. Generally speaking, anything that is implemented without Python loops should be way better than a looped implementation.

But we are not done yet...

## Numpy is Performance King!

Let's implement our function im numpy.
We use here:
* a feature of `norm()` that can take a vector of vectors, 
* numpy function `inner()` that calculates values pariwise,
* broadcasting rules that allows as get a norm matrix of size len(M),len(M)

In [41]:
def cos_matrix_np1(M):
    n = norm(M,axis=1)
    return np.inner(M,M)/(n*n.reshape(-1,1))

In [42]:
s = my_timeit(dict, cos_matrix_np1, 100)

0.085


Once again we make a new breaktrhough in performance.
Let's play a little trick here again. Replace the `norm()` with a `dot()` - this has proved to work better before.

In [43]:
def norm_dot_np(a):
    return np.dot(a,a)

In [44]:
def cos_matrix_np2(M):
    n = np.sqrt(np.apply_along_axis(norm_dot_np,1,M))
    return np.inner(M,M)/(n*n.reshape(-1,1))

In [45]:
s = my_timeit(dict, cos_matrix_np2, 100)

0.067


In [46]:
s[:5]

array([[1.        , 0.03935207, 0.05498267, ..., 0.05346561, 0.04176018,
        0.05739019],
       [0.03935207, 1.        , 0.05328704, ..., 0.04680356, 0.05572377,
        0.03870978],
       [0.05498267, 0.05328704, 1.        , ..., 0.05307461, 0.05030439,
        0.04253165],
       [0.06045495, 0.05640413, 0.05417889, ..., 0.04484612, 0.04931265,
        0.04804116],
       [0.06425379, 0.04670046, 0.06254431, ..., 0.04310505, 0.03313469,
        0.05046435]])

Yep, helped again! 

## Conclusions

* For convinience, use implementations from respected packages.
* When have a multicore machine, use `joblib`
* For speed, use numpy

## Want to see how your systems performed agains ours?

In [47]:
bench1 = OrderedDict([('cos_matrix', 19.840881469969947),
             ('run_cos_matrix_p_threading', 25.139403587207198),
             ('run_cos_matrix_p_multiprocessing', 1.0726150162518024),
             ('cos_matrix_dot_dot', 14.959077935044965),
             ('cos_matrix_dot_dot_cache', 9.097761143619815),
             ('cos_matrix_scipy_pdist', 7.121200116351247),
             ('cos_matrix_sklearn_cossim', 0.31781461276113987),
             ('cos_matrix_sklearn_pd_cosine', 0.32281870637089016),
             ('cos_matrix_np1', 0.17069526184350253),
             ('cos_matrix_np2', 0.09111224306747318)])

In [52]:
bench2 = OrderedDict([('cos_matrix', 12.372514069080353),
             ('run_cos_matrix_p_threading', 101.84953497412305),
             ('run_cos_matrix_p_multiprocessing', 16.79279000808795),
             ('cos_matrix_dot_dot', 8.24763044094046),
             ('cos_matrix_dot_dot_cache', 3.6650888534883657),
             ('cos_matrix_scipy_pdist', 7.646780099719763),
             ('cos_matrix_sklearn_cossim', 0.17296662777662278),
             ('cos_matrix_sklearn_pd_cosine', 0.16396221406757833),
             ('cos_matrix_np1', 0.08530212858691812),
             ('cos_matrix_np2', 0.06693178607150913)])

In [53]:
import pandas as pd
benchmark = pd.DataFrame({"E2690v4 python3.6 vanilla":bench1,
                          "E2690v4 python3.6 anaconda":bench2,
                          "Your System": dict})

In [54]:
benchmark.sort_values(by=benchmark.columns[0], ascending=False)

,E2690v4 python3.6 anaconda,E2690v4 python3.6 vanilla,Your System
run_cos_matrix_p_threading,101.849535,25.139404,101.849535
run_cos_matrix_p_multiprocessing,16.792790,1.072615,16.792790
cos_matrix,12.372514,19.840881,12.372514
cos_matrix_dot_dot,8.247630,14.959078,8.247630
cos_matrix_scipy_pdist,7.646780,7.121200,7.646780
cos_matrix_dot_dot_cache,3.665089,9.097761,3.665089
cos_matrix_sklearn_cossim,0.172967,0.317815,0.172967
cos_matrix_sklearn_pd_cosine,0.163962,0.322819,0.163962
cos_matrix_np1,0.085302,0.170695,0.085302
cos_matrix_np2,0.066932,0.091112,0.066932


# Final thoughts: it's OK to Google it!

Python, scipy and sklearn ecosystem is *huge*. It's virtually impossible to read the whole doc and then say: "Ok, I got it". People google for anwers all time and collect brilliant ideas from Stackoverflow and other sites. 